# Cells inside cells (containers)

Cells must have a unique name.
If you change something in the cell, such as add a label, a grating coupler or just a pin and keep the same name it is very likely that you will have a name conflict when you combine two cells with the same name but different geometry.

To solve this name collision, you can create a new component and add a reference of the old cell into the new cell.
For example, if you can add padding to a Component called 'straight_L3' and you can define a function that returns a new cell that also contains a reference to the original cell.

Cells that have a component argument automatically will copy the component settings into the new cell.

In [ ]:
from typing import Tuple, Optional, List
import gdsfactory as gf


@gf.cell
def add_padding(
    component: gf.Component,
    layers: List[Tuple[int, int]] = [gf.LAYER.DEVREC],
    default: float = 5.0,
    top: Optional[float] = None,
    bottom: Optional[float] = None,
    right: Optional[float] = None,
    left: Optional[float] = None,
) -> gf.Component:
    """Adds padding layers to a container.

    Args:
        component
        layers: list of layers
        suffix for name
        default: default padding
        top: north padding
        bottom: south padding
        right: east padding
        left: west padding
    """

    container = gf.Component()
    container << component

    c = component
    top = top if top else default
    bottom = bottom if bottom else default
    right = right if right else default
    left = left if left else default

    points = [
        [c.xmin - left, c.ymin - bottom],
        [c.xmax + right, c.ymin - bottom],
        [c.xmax + right, c.ymax + top],
        [c.xmin - left, c.ymax + top],
    ]

    for layer in layers:
        container.add_polygon(points, layer=layer)
    return container


wg = gf.components.straight()
wg

In [ ]:
wg_padding = add_padding(component=wg)
wg_padding

In [ ]:
wg.settings

In [ ]:
wg_padding.settings

You can use many containers from gf. Also **note** that many functions have a container version that creates a new cell and a non container version that operates over the cell.

Make sure you only use the function that operate over the cell if you plan to only use that new version of the cell (to avoid name conflicts)

In [ ]:
import gdsfactory as gf

c = gf.components.straight()
c

In [ ]:
c = gf.components.straight()
cc = gf.add_padding(component=c, default=5)
cc

In [ ]:
print(cc.name)  # matches original name

In [ ]:
c = gf.components.straight()
cc = gf.add_padding_container(component=c, default=5)
cc

In [ ]:
print(cc.name)  # new name

In [ ]:
c = gf.components.straight()
cc = gf.components.extension.extend_ports(component=c)
cc

In [ ]:
c = gf.components.spiral_inner_io()
cc = gf.add_grating_couplers.add_grating_couplers_with_loopback_fiber_array(component=c, with_loopback=False)
cc

In [ ]:
c = gf.components.straight()
cc = gf.routing.add_fiber_array(component=c)
cc

In [ ]:
c = gf.components.straight()
cc = gf.routing.add_fiber_single(component=c)
cc

## Composing functions

You can combine more complex functions out of smaller functions.

Lets say that we want to add tapers and grating couplers to a wide waveguide.

In [ ]:
c1 = gf.c.straight()
c1

In [ ]:
straight_wide = gf.partial(gf.components.straight, width=3)
c3 = straight_wide()
c3

In [ ]:
c1 = gf.c.straight(width=3)
c1

In [ ]:
c2 = gf.add_tapers(c1)
c2

In [ ]:
c3 = gf.routing.add_fiber_array(c2, with_loopback=False)
c3

Lets do it with a **single** step thanks to `toolz.pipe`

In [ ]:
import toolz

add_fiber_array = gf.partial(gf.routing.add_fiber_array, with_loopback=False)
add_tapers = gf.add_tapers

# pipe is more readable than the equivalent add_fiber_array(add_tapers(c1))
c3 = toolz.pipe(c1, add_tapers, add_fiber_array)
c3

we can even combine `add_tapers` and `add_fiber_array` thanks to `toolz.compose` or `toolz.compose`

In [ ]:
add_tapers_fiber_array = toolz.compose_left(add_tapers, add_fiber_array)
add_tapers_fiber_array(c1)

In [ ]:
add_tapers_fiber_array = toolz.compose(add_fiber_array, add_tapers)
add_tapers_fiber_array(c1)

In [ ]:
toolz.pipe?

In [ ]:
toolz.compose?